<a href="https://www.kaggle.com/code/karishmabattina/caloriepred-rforest?scriptVersionId=240816441" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
train_data = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test_data = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")

In [3]:
train_copy, test_copy = train_data, test_data

In [4]:
train_data.sample(5)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Calories
210925,210925,male,47,176.0,78.0,29.0,113.0,40.9,219.0
390730,390730,male,40,181.0,81.0,22.0,113.0,40.7,164.0
742543,742543,female,21,157.0,52.0,18.0,97.0,40.6,93.0
238761,238761,male,35,192.0,91.0,5.0,86.0,39.0,17.0
157818,157818,male,28,184.0,85.0,26.0,114.0,40.8,189.0


In [5]:
test_data.sample(5)

,id,Sex,Age,Height,Weight,Duration,Heart_Rate,Body_Temp
11495,761495,male,24,171.0,77.0,1.0,81.0,37.8
163928,913928,male,36,182.0,90.0,20.0,98.0,40.7
213064,963064,female,29,159.0,58.0,4.0,86.0,38.9
193708,943708,male,63,189.0,97.0,14.0,89.0,40.3
22486,772486,male,45,175.0,79.0,8.0,84.0,39.9


In [6]:
train_data.shape

(750000, 9)

In [7]:
test_data.shape

(250000, 8)

In [8]:
train_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
Calories      0
dtype: int64

In [9]:
test_data.isnull().sum()

id            0
Sex           0
Age           0
Height        0
Weight        0
Duration      0
Heart_Rate    0
Body_Temp     0
dtype: int64

In [10]:
def feature_engineering(df):
    df['bmi'] = df['Weight'] / ((df['Height'] / 100) ** 2)    
    df['exercise_intensity'] = df['Heart_Rate'] / df['Duration']
    df['heart_rate_duration'] = df['Heart_Rate'] * df['Duration']
    df['temp_duration'] = df['Body_Temp'] * df['Duration']
    df['hr_to_temp'] = df['Heart_Rate'] / df['Body_Temp']
    df['hr_to_age'] = df['Heart_Rate'] / df['Age']
    df['age_bmi'] = df['Age'] * df['bmi']
    df['max_heart_rate'] = 220 - df['Age']
    df['heart_rate_intensity'] = df['Heart_Rate'] / df['max_heart_rate']
    return df

In [11]:
train_data = feature_engineering(train_data)
test_data = feature_engineering(test_data)

In [12]:
gender_map = {"male" : 0,"female" : 1}
train_data['Sex'] = train_data['Sex'].replace(gender_map)

train_data=train_data.drop(["id"],axis=1)

/tmp/ipykernel_13/4161382762.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_data['Sex'] = train_data['Sex'].replace(gender_map)


In [13]:
X = train_data.drop(['Calories'], axis=1)
y = train_data['Calories']

In [14]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import r2_score, mean_squared_error

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [16]:
rf_params = {'n_jobs': -1,
             'n_estimators': 500,
             'min_samples_split': 2,
             'min_samples_leaf': 2,
             'max_features': 'log2',
             'max_depth': 20}

In [17]:
rf_model = RandomForestRegressor(**rf_params)
rf_model.fit(X_train, y_train)
rf_model.score(X_test, y_test)
y_pred = rf_model.predict(X_test)
print(f"RandomForest → R²: {r2_score(y_test, y_pred):.4f}, RMSE: {mean_squared_error(y_test, y_pred, squared=False):.2f}")

RandomForest → R²: 0.9965, RMSE: 3.66


In [18]:
test_data['Sex'] = test_data['Sex'].replace(gender_map)
test_ids = test_data['id']
X_test_final = test_data.drop(["id"], axis=1)


test_predictions = rf_model.predict(X_test_final)

submission = pd.DataFrame({
    'id': test_ids,
    'Calories': test_predictions
})
submission.to_csv('submission.csv', index=False)

/tmp/ipykernel_13/1108437728.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  test_data['Sex'] = test_data['Sex'].replace(gender_map)


In [19]:
submission

,id,Calories
0,750000,27.287540
1,750001,107.054332
2,750002,87.751680
3,750003,128.025272
4,750004,75.623381
...,...,...
249995,999995,26.231669
249996,999996,9.214131
249997,999997,73.287732
249998,999998,168.819011
